<a href="https://colab.research.google.com/github/tcglarry/geo/blob/master/geo_stage_2_11x11x10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -*- coding: utf-8 -*-
"""ali_build_model_stage_2_MobileNetV2_aug_1000_dense.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wSVEmcoWzRm-7B6UGkHB8ysfMDRT_xK5
"""

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls 'drive/My Drive/geo/'

geo_stage2_first_try.h5		Untitled1.ipynb
geo_Unet_colab.ipynb		Untitled2.ipynb
geo_Unet.ipynb			upload_sample.csv
test2.npy			water_first_try_2_2.h5
test_submit_0831_2.csv		water_first_try_2.h5
test_submit_0831_3.csv		water_first_try_2_prepro_ch16.h5
test_submit_0831_ch32.csv	water_first_try_2_prepro_ch32_gram.h5
test_submit_0831_ch32_gram.csv	water_first_try_2_prepro_ch32.h5
test_submit_0831_ch64_gram.csv	water_first_try_2_prepro_ch64_gram.h5
test_submit_0831.csv		water_first_try_2_prepro.h5
test_submit.csv			water_first_try_2_prepro_twoloss.h5
train_data.npy			water_first_try_3_prepro.h5
train_labels.npy		water_first_try.h5
Untitled0.ipynb


In [0]:
src=  'drive/My Drive/geo/'

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Conv2DTranspose,BatchNormalization
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers

In [11]:
data= np.load(src+'train_data.npy')
label = np.load(src+'train_labels.npy')
label = label[:,:,:,np.newaxis]
print (data.shape)
print (label.shape)



(4200, 21, 21, 20)
(4200, 20, 20, 1)


# Reduce H,W from 21 to 11, Reduce channel from 21 to 11





In [5]:
index_list_2 = list(range(20))
select_list_2 = [idx for idx in index_list_2 if idx % 2  == 0 ]

index_list_2 = list(range(21))
select_list_0 = [idx for idx in index_list_2 if idx % 2  == 0 ]
print (select_list_2)
print (select_list_0)



[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [13]:
data = data[:,:,:,select_list_2]

data = data[:,select_list_0,:,:]

data = data[:,:,select_list_0,:]
data.shape

(4200, 11, 11, 10)

In [0]:
data_mean = np.mean(data)
data_std = np.std(data)
label_max = np.max(label)
label_min = np.min(label)

train_data = (data - data_mean)/data_std
#label_data = (label-label_min)/(label_max - label_min)
label_data = np.log(label)

In [0]:
train_x,val_x,train_y,val_y = train_test_split(train_data,label_data,test_size=0.2, shuffle=True)

In [0]:
def conv_block(ch, activation= 'relu', padding='same',kernel_regularizer=regularizers.l2(0.01)):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )

In [0]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=8):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((11,11,10))
    
    up_1 = UpSampling2D(size=(2,2))(inputs)

    conv0 = Conv2D(32,(3,3),padding='valid')(up_1)
    print ('conv0',conv0.get_shape())
    conv1 =  conv_block(ch)(conv0)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    print ('pool1',pool1.get_shape())
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    print ('poo12',pool2.get_shape())
    
    conv_test = Conv2D(64,(2,2),padding='valid')(pool2)
    print ('conv_test',conv_test.get_shape())
    
    
    
    conv3 =  conv_block(ch*4)(conv_test)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    print ('poo13',pool3.get_shape())
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    print ('poo14',pool4.get_shape())
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    print ('conv7',conv7.get_shape())
    conv7 = Conv2DTranspose(ch*4,(2,2),padding='valid')(conv7)
    print ('conv7',conv7.get_shape())
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
  
    model = Model(inputs= inputs, outputs=conv10)
    model.summary()
    
    return model
    
    

In [20]:
model = build_model_1()

conv0 (?, 20, 20, 32)
pool1 (?, 10, 10, 8)
poo12 (?, 5, 5, 16)
conv_test (?, 4, 4, 64)
poo13 (?, 2, 2, 32)
poo14 (?, 1, 1, 64)
conv7 (?, 4, 4, 32)
conv7 (?, ?, ?, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 11, 11, 10)   0                                            
__________________________________________________________________________________________________
up_sampling2d_2 (UpSampling2D)  (None, 22, 22, 10)   0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 20, 20, 32)   2912        up_sampling2d_2[0][0]            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 20

In [0]:
model.compile (optimizer='Adam', loss = 'mse', metrics =['mae'])

In [22]:

# train the model on the new data for a few epochs

use_model= False
if use_model:
    

    if os.path.isfile(src+'water_first_try.h5'):

      model = load_model(src+'water_first_try.h5')
      print ('model laoded')
    else:
      print ('model not exist')
else:
  print ('not to use model')


not to use model


In [23]:





checkpoint = ModelCheckpoint(filepath=src+'geo_stage2_first_try_11_11_10.h5', monitor = 'val_loss', save_best_only=True, mode= 'auto')
earlystop = EarlyStopping(patience=30)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.00001,verbose=1)
callback_list = [checkpoint, earlystop,reduce_lr]
model.fit(x=train_x, y=train_y, batch_size=32, epochs=1000, callbacks = callback_list, verbose=1, validation_split=0.2,  shuffle=True)

Train on 2688 samples, validate on 672 samples
Epoch 1/1000
2688/2688 [==============================] - 5s 2ms/step - loss: 0.8644 - mean_absolute_error: 0.7322 - val_loss: 0.5979 - val_mean_absolute_error: 0.6109
Epoch 2/1000
2688/2688 [==============================] - 2s 885us/step - loss: 0.5026 - mean_absolute_error: 0.5581 - val_loss: 0.4319 - val_mean_absolute_error: 0.5158
Epoch 3/1000
2688/2688 [==============================] - 2s 895us/step - loss: 0.3862 - mean_absolute_error: 0.4868 - val_loss: 0.3464 - val_mean_absolute_error: 0.4603
Epoch 4/1000
2688/2688 [==============================] - 2s 880us/step - loss: 0.3319 - mean_absolute_error: 0.4510 - val_loss: 0.3297 - val_mean_absolute_error: 0.4526
Epoch 5/1000
2688/2688 [==============================] - 2s 890us/step - loss: 0.3013 - mean_absolute_error: 0.4297 - val_loss: 0.2929 - val_mean_absolute_error: 0.4218
Epoch 6/1000
2688/2688 [==============================] - 2s 894us/step - loss: 0.2785 - mean_absolute_er

In [24]:

error = model.evaluate(val_x,val_y)
print (error)

840/840 [==============================] - 0s 272us/step
[0.1610054027466547, 0.31099959924107506]


In [0]:
model.evaluate(val_x,val_y)

with open('train_data.pickle', 'wb') as f:
    pickle.dump(train_data, f)

with open('train_data.pickle', 'rb') as f:
    train_data = pickle.load(f)

print (train_data.shape)    
